# Alexnet 구현

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, Layer
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras import backend as K

Using TensorFlow backend.


In [0]:
from tensorflow.keras.utils import to_categorical

In [0]:
x_train = x_train.reshape(227,227,3)
x_test = x_test.reshape(227,227,3)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

### Architecture
1. 활성화함수 ReLU 
2. Local Response Normalization
    - ReLU 활성화 함수로 인해 양수 방향으로 무한히 커질 가능성이 있고,    
      너무 큰 값이 주변 값들을 무시하게 할 수도 있기 때문에 정규화과정을 수행
    - 코드 출처: https://datascienceschool.net/view-notebook/d19e803640094f76b93f11b850b920a4/ 
3. Overlapping Pooling 
4. Data Augmentation
    - generating image translations and horizontal reflections
    - perform PCA 
5. Dropout 
    - probability 0.5
6. SGD Optimizer
    - batch size: 128
    - momentum: 0.9
    - weight decay: 0.0005

In [0]:
class LocalResponseNormalization(Layer):

    def __init__(self, n=5, alpha=1e-4, beta=0.75, k=2, **kwargs):
        self.n = n
        self.alpha = alpha
        self.beta = beta
        self.k = k
        super(LocalResponseNormalization, self).__init__(**kwargs)

    def build(self, input_shape):
        self.shape = input_shape
        super(LocalResponseNormalization, self).build(input_shape)

    def call(self, x):
        _, r, c, f = self.shape 
        squared = K.square(x)
        pooled = K.pool2d(squared, (self.n, self.n), strides=(1,1), padding="same", pool_mode='avg')
        summed = K.sum(pooled, axis=3, keepdims=True)
        averaged = self.alpha * K.repeat_elements(summed, f, axis=3)
        denom = K.pow(self.k + averaged, self.beta)
        return x / denom 
    
    def compute_output_shape(self, input_shape):
        return input_shape 

In [3]:
model = Sequential()

# layer1: Convolutional Layer 1
model.add(Conv2D(filters=96, input_shape=(227,227,3), 
                 kernel_size=(11,11), strides=(4,4), padding='valid'))
model.add(Activation('relu')) # 활성함수 relu
model.add(LocalResponseNormalization(input_shape=model.output_shape[1:]))

# layer2: Max Pooling 1 
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
                        # padding 'valid': 유효한 영역만 출력됨. 출력 이미지 사이즈 < 입력 사이즈 
                        # padding 'same': 출력 이미지 사이즈 = 입력 사이즈 

        
# layer3: Convolutional Layer 2
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same'))
model.add(Activation('relu'))
model.add(LocalResponseNormalization(input_shape=model.output_shape[1:]))

# layer4: Max Pooling 2
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))


# layer5: Convolutional Layer 3
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))

# layer6: Convolutional Layer 4
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))

# layer7: Convolutional Layer 5
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))

# layer8: Max Pooling 3
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))


# To Fully Connected layer
model.add(Flatten())


# layer9: Fully Connected Layer 1 
model.add(Dense(4096, input_shape=(6*6*256,)))
model.add(Activation('relu'))
model.add(Dropout(0.5)) # Dropout

# layer10: Fully Connected Layer 2 
model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.5)) # Dropout

# layer11: Fully Connected Layer 3 
model.add(Dense(1000)) # train dataset의 크기 N
model.add(Activation('relu'))

# Output Layer
model.add(BatchNormalization())
model.add(Dense(10)) # categorical output의 개수 
model.add(Activation('softmax'))







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
optimizer = optimizers.SGD(lr=0.01, decay=5e-5, momentum=0.9)

In [5]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

![AlexNet3](https://user-images.githubusercontent.com/43749571/77500008-18002080-6e97-11ea-93ee-aa924eb69bac.png)

In [6]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_1 (Activation)    (None, 55, 55, 96)        0         
_________________________________________________________________
local_response_normalization (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
activation_2 (Activation)    (None, 27, 27, 256)       0         
_________________________________________________________________
local_response_normalization (None, 27, 27, 256)      

In [0]:
history = model.fit(x_train, y_train, epochs=50, batch_size=128, verbose=2)